Código em Python, responsável por realizar o agrupamento semântico de uma lista de aspectos referentes a dados extraídos de review reais sobre laptops contida em um arquivo de entrada .CSV.

Importação das bibliotecas utilizadas:


In [0]:
import numpy as np
import pandas as pd
import spacy as sp
import nltk
import re
from google.colab import drive
import string


Carregamento do arquivo de entrada (lista de aspectos) e formatação:

In [3]:
# Utilizando o drive do usuário: 
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open('/content/drive/My Drive/laptop_filtered_aspect_sample.csv', mode='r', encoding='utf-8') as f:
  listaAspectos = f.read()

In [0]:
listaAspectos = listaAspectos.split('\n')

In [6]:
# Elimina o cabeçalho (item cujo índice é zero e que dá nome à coluna do arquivo 
# e não repensenta um dado de fato, isto é, não é um "aspecto")
listaAspectos.pop(0)

'aspect_name'

In [7]:
# Conferindo...
print(len(listaAspectos))

7335


In [8]:
print(listaAspectos)

['laptop', 'use', 'computer', 'screen', 'price', 'keyboard', 'work', 'battery', 'product', 'fast', 'time', 'battery life', 'notebook', 'easy', 'purchase', 'works', 'performance', 'com', 'machine', 'looking', 'chromebook', 'very good', 'quality', 'money', 'light', 'windows', 'little', 'speed', 'size', 'run', 'problem', 'apps', 'games', 'internet', 'promotion', 'school', 'display', 'issues', 'review was', 'hours', 'gaming', 'set', 'problems', 'tablet', 'os', 'device', 'design', 'memory', 'day', 'ram', 'daughter', 'very happy', 'software', 'power', 'runs', 'hard drive', 'programs', 'update', 'laptops', 'features', 'return', 'touch screen', 'issue', 'system', 'college', 'needs', 'storage', 'look', 'great laptop', 'pc', 'processor', 'model', 'keys', 'sound', 'weight', 'ssd', 'box', 'lightweight', 'usb', 'start', 'charge', 'drive', 'fan', 'speakers', 'great product', 'touchscreen', 'very fast', 'bit', 'upgrade', 'son', 'home', 'cost', 'long', 'light weight', 'slow', 'products', 'videos', 'wa

Separação da lista. O arquivo possui termos simples (uma única palavra) e compostos (formados por duas palavras). Para facilitar a análise e o respectivo agrupamento semântico é realizado tal separação. Além disso, há termos grafados em português na lista que é majoritariamente composta por palavras escritas em Inglês. Assim, é feita essa separação também.

In [0]:
# Função que retorna duas listas (as formadas por palavras simples e as compostas):
def separaLista(lista):
  listaAuxSimples = []
  listaAuxCompostas = []
  for expressao in lista:
    if re.search(' ', expressao):
      listaAuxCompostas.append(expressao)
    else:
      listaAuxSimples.append(expressao)
  return listaAuxSimples, listaAuxCompostas 

In [0]:
# Separa-se o arquivo em duas listas distintas:
listaAspectosSimples, listaAspectosCompostos = separaLista(listaAspectos)

In [11]:
# Conferindo...
print(len(listaAspectosSimples))
print(listaAspectosSimples)
print(len(listaAspectosCompostos))
print(listaAspectosCompostos)

2846
['laptop', 'use', 'computer', 'screen', 'price', 'keyboard', 'work', 'battery', 'product', 'fast', 'time', 'notebook', 'easy', 'purchase', 'works', 'performance', 'com', 'machine', 'looking', 'chromebook', 'quality', 'money', 'light', 'windows', 'little', 'speed', 'size', 'run', 'problem', 'apps', 'games', 'internet', 'promotion', 'school', 'display', 'issues', 'hours', 'gaming', 'set', 'problems', 'tablet', 'os', 'device', 'design', 'memory', 'day', 'ram', 'daughter', 'software', 'power', 'runs', 'programs', 'update', 'laptops', 'features', 'return', 'issue', 'system', 'college', 'needs', 'storage', 'look', 'pc', 'processor', 'model', 'keys', 'sound', 'weight', 'ssd', 'box', 'lightweight', 'usb', 'start', 'charge', 'drive', 'fan', 'speakers', 'touchscreen', 'bit', 'upgrade', 'son', 'home', 'cost', 'long', 'slow', 'products', 'videos', 'warranty', 'mouse', 'hardware', 'specs', 'value', 'type', 'touchpad', 'graphics', 'review', 'gb', 'unit', 'delivery', 'chrome', 'computers', 'touc

Tomando a lista de aspectos formadas por palavras únicas objetiva-se agora separar as palavras grafadas em portugês das em inglês. Para a tarefa são utilizadas as bibliotecas multilingual do NLTK (Natural Language Tool Kit) e do spaCy. Elas possuem diversas ferramentas para manipulação, tratamento de bases e outros recursos junto a um extenso dicionário embutido (no caso do NLTK).

In [16]:
# Importação dos modelos necessários para a tarefa de separação:
nltk.download('wordnet')
nltk.download('omw') # Open Multilingual WordNet
nltk.download('stopwords')
from nltk.stem import PorterStemmer # usada p/ extrair radical da palavra
!python3 -m spacy download pt_core_news_sm
import pt_core_news_sm
!python3 -m spacy download en_core_web_lg
import en_core_web_lg

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
# Carregamento da base de corpus (é utilizada aqui a wordnet que trata-se de uma 
# grande e abrangente banco de dados léxico). Lista de línguas que podem ser
# utilizadas: ("eng", "ind", "zsm", "jpn", "tha", "cmn", "qcn", "fas", "arb",
# "heb", "ita", "por", "nob", "nno", "dan", "swe", "fra", "fin", "ell", "glg",
# "cat", "spa", "eus", "als", "pol", "slv")
vocabPortugues = set(w.lower() for w in nltk.corpus.wordnet.words('por'))
vocabIngles = set(w.lower() for w in nltk.corpus.wordnet.words('eng'))

In [0]:
# Função que separa a lista de aspectos simples retornando 3 listas (português, 
# inglês e a de palavras não reconheciadas pelos dicionários da base):
def obtemListasPortIng(lista):
  listaPortAux = []
  listaIngAux = []
  listaPalavrasNaoReconAux = []
  nlp = pt_core_news_sm.load()
  for termo in lista:
    w = []
    doc = nlp(termo)
    w = [token.lemma_ for token in doc]
    if ((w[0] in vocabPortugues) or (nltk.PorterStemmer().stem(termo)) in vocabPortugues) and (nltk.corpus.wordnet.morphy(termo) not in vocabIngles):
      listaPortAux.append(termo)
    else:
      if ((w[0] not in vocabPortugues) or (nltk.PorterStemmer().stem(termo)) not in vocabPortugues) and (nltk.corpus.wordnet.morphy(termo) not in vocabIngles):
        listaPalavrasNaoReconAux.append(termo)
      else:
        listaIngAux.append(termo)
  return listaPortAux, listaIngAux, listaPalavrasNaoReconAux

In [0]:
# Obtem-se 3 novas listas (português, inglês e a de termos não encontrados):
listaPortugues, listaIngles, listaPalavrasNaoRecon = obtemListasPortIng(listaAspectosSimples)

In [20]:
# Conferindo...
print(len(listaPortugues))
print(listaPortugues)

194
['com', 'entrega', 'prazo', 'ótimo', 'recomendo', 'produto', 'tela', 'bateria', 'é', 'computador', 'sim', 'compra', 'mais', 'muito', 'geral', 'qualidade', 'todos', 'por', 'não', 'previsto', 'garantia', 'estado', 'trabalho', 'troca', 'necessidade', 'preço', 'declarar', 'achei', 'deu', 'vaio', 'outro', 'melhor', 'aparelho', 'única', 'uso', 'leves', 'designe', 'opção', 'aquisição', 'resto', 'mesmo', 'reais', 'desejar', 'configuração', 'desempenho', 'faz', 'anunciado', 'atendeu', 'memória', 'benefício', 'reclamar', 'tem', 'começo', 'velocidade', 'caixa', 'robusto', 'deixou', 'geração', 'sims', 'ter', 'retirada', 'bons', 'finalidade', 'tarefas', 'máquina', 'rápido', 'trocar', 'demais', 'mês', 'utilizar', 'rapidez', 'restante', 'positivo', 'fraco', 'esperado', 'empresa', 'diferença', 'está', 'chegou', 'ele', 'venda', 'instalação', 'jogos', 'descrição', 'pena', 'etc', 'anúncio', 'também', 'recursos', 'pc’s', 'nele', 'detalhes', 'segundos', 'teclado', 'falta', 'custo', 'perfeito', 'externo

In [21]:
# Conferindo...
print(len(listaIngles))
print(listaIngles)

2389
['laptop', 'use', 'computer', 'screen', 'price', 'keyboard', 'work', 'battery', 'product', 'fast', 'time', 'notebook', 'easy', 'purchase', 'works', 'performance', 'machine', 'looking', 'quality', 'money', 'light', 'windows', 'little', 'speed', 'size', 'run', 'problem', 'games', 'internet', 'promotion', 'school', 'display', 'issues', 'hours', 'gaming', 'set', 'problems', 'tablet', 'os', 'device', 'design', 'memory', 'day', 'ram', 'daughter', 'software', 'power', 'runs', 'programs', 'update', 'laptops', 'features', 'return', 'issue', 'system', 'college', 'needs', 'storage', 'look', 'pc', 'processor', 'model', 'keys', 'sound', 'weight', 'box', 'lightweight', 'start', 'charge', 'drive', 'fan', 'speakers', 'touchscreen', 'bit', 'upgrade', 'son', 'home', 'cost', 'long', 'slow', 'products', 'videos', 'warranty', 'mouse', 'hardware', 'specs', 'value', 'type', 'graphics', 'review', 'gb', 'unit', 'delivery', 'chrome', 'computers', 'touch', 'ports', 'install', 'download', 'small', 'support',

In [22]:
# Conferindo...
print(len(listaPalavrasNaoRecon))
print(listaPalavrasNaoRecon)

263
['chromebook', 'apps', 'ssd', 'usb', 'touchpad', 'trackpad', 'chromebooks', 'hdd', 'app', 'hd', 'i7', 'macbook', 'bios', 'i5', 'bluetooth', 'hdmi', 'bloatware', 'netflix', 'motherboard', 'lenovo', 'backlit', 'macbooks', 'x1', 'i3', 'mx150', 'multitasking', 'walmart', 'samsung', 'backlight', 'wi-fi', 'thinkpad', 'smartphone', 'brainer', 'gpu', 'refurb', 'netbook', 'gamers', 'que', 'youtube', 'skype', 're-', 'malware', 'cost-benefit', 'bootup', 'chromeos', 'antivirus', 'mic', 'powerpoint', 'vga', 'intel', 'iphones', 'login', 'ultrabook', 'minecraft', 'facebook', 'nvme', 'itunes', 'applecare', 'harddrive', 'touchpads', 'autocad', '60fps', 'c302', 'asus', '1080p', 'c.', 'p52s', 'pcie', 'vezes', 'internals', 'sata', 'chipset', 'magalu', 'thinkpads', 'touchbar', 'playstore', 'ipad', 'ctrl', 'labtop', 'xp', 'pixelbook', 'workaround', 's.', 'high-end', 'approx', 'grandkids', 'notbook', 'anti-', 'ipads', 'aftermarket', 'iphone', 'overwatch', 'prós', 'i9', 'gameplay', '1050ti', 'turbo', 'gma

In [0]:
# Função que remove as stopwords da lista de palavras em português:
def removeStopWordsPort(lista):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    listaAux = []
    listaAuxStopwords = []    
    for item in lista:
        if item not in stopwords:
            listaAux.append(item)
        else:
            listaAuxStopwords.append(item)
    return listaAux, listaAuxStopwords

In [0]:
# Função que remove as stopwords da lista de palavras em inglês:
def removeStopWordsIng(lista):
    stopwords = nltk.corpus.stopwords.words('english')
    listaAux = []
    listaAuxStopwords = []
    for item in lista:
        if item not in stopwords:
            listaAux.append(item)
        else:
            listaAuxStopwords.append(item)
    return listaAux, listaAuxStopwords

In [0]:
# Retira-se das listas de palavras em inglês e português as chamadas stopwords 
# que são palavras que não agregam muito valor para a análise do processamento 
# de linguagem natural (preposições, artigos, conjunções, por exemplo): 
listaPortugues, listaStopwordsPort = removeStopWordsPort(listaPortugues)
listaIngles, listaStopwordsIng = removeStopWordsIng(listaIngles)
# Pesquisa-se se na lista das "não encontradas" existe stopwords:
listaPalavrasNaoRecon, listaStopwordsNaoRecon = removeStopWordsPort(listaPalavrasNaoRecon) 

In [33]:
# Conferindo...
print(len(listaPortugues))
print(listaPortugues)
print(listaStopwordsPort)

178
['entrega', 'prazo', 'ótimo', 'recomendo', 'produto', 'tela', 'bateria', 'computador', 'sim', 'compra', 'geral', 'qualidade', 'todos', 'previsto', 'garantia', 'estado', 'trabalho', 'troca', 'necessidade', 'preço', 'declarar', 'achei', 'deu', 'vaio', 'outro', 'melhor', 'aparelho', 'única', 'uso', 'leves', 'designe', 'opção', 'aquisição', 'resto', 'reais', 'desejar', 'configuração', 'desempenho', 'faz', 'anunciado', 'atendeu', 'memória', 'benefício', 'reclamar', 'começo', 'velocidade', 'caixa', 'robusto', 'deixou', 'geração', 'sims', 'ter', 'retirada', 'bons', 'finalidade', 'tarefas', 'máquina', 'rápido', 'trocar', 'demais', 'mês', 'utilizar', 'rapidez', 'restante', 'positivo', 'fraco', 'esperado', 'empresa', 'diferença', 'chegou', 'venda', 'instalação', 'jogos', 'descrição', 'pena', 'etc', 'anúncio', 'recursos', 'pc’s', 'nele', 'detalhes', 'segundos', 'teclado', 'falta', 'custo', 'perfeito', 'externo', 'lazer', 'placa', 'três', 'problemas', 'carcaça', 'capacidade', 'exemplo', 'excel

In [27]:
# Conferindo...
print(len(listaIngles))
print(listaIngles)
print(listaStopwordsIng)

2381
['laptop', 'use', 'computer', 'screen', 'price', 'keyboard', 'work', 'battery', 'product', 'fast', 'time', 'notebook', 'easy', 'purchase', 'works', 'performance', 'machine', 'looking', 'quality', 'money', 'light', 'windows', 'little', 'speed', 'size', 'run', 'problem', 'games', 'internet', 'promotion', 'school', 'display', 'issues', 'hours', 'gaming', 'set', 'problems', 'tablet', 'os', 'device', 'design', 'memory', 'day', 'ram', 'daughter', 'software', 'power', 'runs', 'programs', 'update', 'laptops', 'features', 'return', 'issue', 'system', 'college', 'needs', 'storage', 'look', 'pc', 'processor', 'model', 'keys', 'sound', 'weight', 'box', 'lightweight', 'start', 'charge', 'drive', 'fan', 'speakers', 'touchscreen', 'bit', 'upgrade', 'son', 'home', 'cost', 'long', 'slow', 'products', 'videos', 'warranty', 'mouse', 'hardware', 'specs', 'value', 'type', 'graphics', 'review', 'gb', 'unit', 'delivery', 'chrome', 'computers', 'touch', 'ports', 'install', 'download', 'small', 'support',

In [28]:
# Conferindo...
print(len(listaPalavrasNaoRecon))
print(listaPalavrasNaoRecon)
print(listaStopwordsNaoRecon)

258
['chromebook', 'apps', 'ssd', 'usb', 'touchpad', 'trackpad', 'chromebooks', 'hdd', 'app', 'hd', 'i7', 'macbook', 'bios', 'i5', 'bluetooth', 'hdmi', 'bloatware', 'netflix', 'motherboard', 'lenovo', 'backlit', 'macbooks', 'x1', 'i3', 'mx150', 'multitasking', 'walmart', 'samsung', 'backlight', 'wi-fi', 'thinkpad', 'smartphone', 'brainer', 'gpu', 'refurb', 'netbook', 'gamers', 'youtube', 'skype', 're-', 'malware', 'cost-benefit', 'bootup', 'chromeos', 'antivirus', 'mic', 'powerpoint', 'vga', 'intel', 'iphones', 'login', 'ultrabook', 'minecraft', 'facebook', 'nvme', 'itunes', 'applecare', 'harddrive', 'touchpads', 'autocad', '60fps', 'c302', 'asus', '1080p', 'c.', 'p52s', 'pcie', 'vezes', 'internals', 'sata', 'chipset', 'magalu', 'thinkpads', 'touchbar', 'playstore', 'ipad', 'ctrl', 'labtop', 'xp', 'pixelbook', 'workaround', 's.', 'high-end', 'approx', 'grandkids', 'notbook', 'anti-', 'ipads', 'aftermarket', 'iphone', 'overwatch', 'prós', 'i9', 'gameplay', '1050ti', 'turbo', 'gmail', 'l

In [29]:
# Ao inspecionar a lista de palavras não reconhecidas nota-se que muitos termos 
# são utilizados no linguajar técnico da área de informática/tecnologia. 
# Sendo assim, vamos submeter a lista à um outro dicionário para extrair mais 
# palavras para a lista das palavras em inglês (a base "webtext" corresponde
# a coleção de textos retirados da web como do fórum de discussão da Firefox, 
# por exemplo): 
nltk.download('webtext')
vocabIngles2 = set(w for w in nltk.corpus.webtext.words())

[nltk_data] Downloading package webtext to /root/nltk_data...
[nltk_data]   Unzipping corpora/webtext.zip.


In [0]:
# Rotina que recupera palavras da lista de não reconhecidas
listaPalavrasRecuperadas = []
listaPalavrasNaoReconFinal = []
for item in listaPalavrasNaoRecon:
  if item in vocabIngles2:
    listaPalavrasRecuperadas.append(item)
    listaIngles.append(item)
  else:
    listaPalavrasNaoReconFinal.append(item)

In [31]:
print(len(listaPalavrasRecuperadas))
print(listaPalavrasRecuperadas)

45
['apps', 'touchpad', 'trackpad', 'hdd', 'app', 'hd', 'motherboard', 'multitasking', 'thinkpad', 'antivirus', 'mic', 'login', 'harddrive', 'ctrl', 'xp', 'approx', 'turbo', 'uninstall', 'customization', 'pdfs', 'osx', 'pdf', 'taskbar', '+', 'fn', 'plugins', 'config', 'spacebar', 'plugin', 'username', 'biggie', 'fiancé', 'coworkers', 'hotkeys', 'workarounds', 'coworker', 'installer', 'everytime', 'bestbuy', 'toolbar', 'ui', 'offline', 'mah', 'gon', 'popups']


In [32]:
print(len(listaPalavrasNaoReconFinal))
print(listaPalavrasNaoReconFinal)

213
['chromebook', 'ssd', 'usb', 'chromebooks', 'i7', 'macbook', 'bios', 'i5', 'bluetooth', 'hdmi', 'bloatware', 'netflix', 'lenovo', 'backlit', 'macbooks', 'x1', 'i3', 'mx150', 'walmart', 'samsung', 'backlight', 'wi-fi', 'smartphone', 'brainer', 'gpu', 'refurb', 'netbook', 'gamers', 'youtube', 'skype', 're-', 'malware', 'cost-benefit', 'bootup', 'chromeos', 'powerpoint', 'vga', 'intel', 'iphones', 'ultrabook', 'minecraft', 'facebook', 'nvme', 'itunes', 'applecare', 'touchpads', 'autocad', '60fps', 'c302', 'asus', '1080p', 'c.', 'p52s', 'pcie', 'vezes', 'internals', 'sata', 'chipset', 'magalu', 'thinkpads', 'touchbar', 'playstore', 'ipad', 'labtop', 'pixelbook', 'workaround', 's.', 'high-end', 'grandkids', 'notbook', 'anti-', 'ipads', 'aftermarket', 'iphone', 'overwatch', 'prós', 'i9', 'gameplay', '1050ti', 'gmail', 'lappy', 'trackpads', 't480s', 'quadcore', 'netbooks', 'desing', 'earbuds', 'xps', 'cb', 'ssds', 'solidworks', 'mbps', 'usd', 'chromecast', 'aero', 'upgradability', 'top-of

Agora que temos todas as listas separadas faremos o agrupamento dos termos semelhantes. Para a tarefa é empregada a propriedade "similarity" da biblioteca spaCy que avalia a similaridade semântica estimada entre as palavras.

In [0]:
# Obtem os grupos de palavras semelhantes para as palavras em inglês:
nlp = en_core_web_lg.load()
listaInglesAux = [item for item in listaIngles]
listaInglesAux2 = [item for item in listaIngles]  
for item in listaInglesAux:
  grupoAux = []
  token1 = nlp(item)
    for item2 in listaInglesAux2:
      token2 = nlp(item2)
      if (token1.similarity(token2) > 0.6):
        grupoAux.append(item2)
  del listaInglesAux2[0]
  if (grupoAux):
    arquivo = open('/content/drive/My Drive/Grupos_palavras_ingles.txt', 'a')
    arquivo.write(str(grupoAux)) 
    arquivo.write('\n')
    arquivo.close()

In [0]:
# Obtem os grupos de palavras semelhantes para os termos compostos
# (aqueles termos que mantém mais de uma palavra no arquivo original)
nlp = en_core_web_lg.load()
listaAspectosCompostosAux = [item for item in listaAspectosCompostos]
listaAspectosCompostosAux2 = [item for item in listaAspectosCompostos]  
for item in listaAspectosCompostosAux:
  grupoAux = []
  token1 = nlp(item)
  for item2 in listaAspectosCompostosAux2:
    token2 = nlp(item2)
    if (token1.similarity(token2) > 0.8):
      grupoAux.append(item2)
  del listaAspectosCompostosAux2[0]
  if (grupoAux):
    arquivo = open('/content/drive/My Drive/Grupos_palavras_compostas.txt', 'a')
    arquivo.write(str(grupoAux)) 
    arquivo.write('\n')
    arquivo.close()

In [0]:
# Obtem os grupos de palavras semelhantes para as palavras "não reconhecidas"
# pelos dicionários
nlp = en_core_web_lg.load()
listaPalavrasNaoReconFinalAux = [item for item in listaPalavrasNaoReconFinal]
listaPalavrasNaoReconFinalAux2 = [item for item in listaPalavrasNaoReconFinal]  
for item in listaPalavrasNaoReconFinalAux:
  grupoAux = []
  token1 = nlp(item)
  for item2 in listaPalavrasNaoReconFinalAux2:
    token2 = nlp(item2)
    if (token1.similarity(token2) > 0.6):
      grupoAux.append(item2)
  del listaPalavrasNaoReconFinalAux2[0]
  if (grupoAux):
    arquivo = open('/content/drive/My Drive/Grupos_palavras_nao_encontradas.txt', 'a')
    arquivo.write(str(grupoAux)) 
    arquivo.write('\n')
    arquivo.close()

In [0]:
# Obtem os grupos de palavras semelhantes para as palavras em português:
nlp = pt_core_news_sm.load()
listaPortuguesAux = [item for item in listaPortugues]
listaPortuguesAux2 = [item for item in listaPortugues]
for item in listaPortuguesAux:
  grupoAux = []
  token1 = nlp(item)
  for item2 in listaPortuguesAux2:
    token2 = nlp(item2)
    if (token1.similarity(token2) > 0.8):
      grupoAux.append(item2)
  del listaPortuguesAux2[0]
  if (grupoAux):
    arquivo = open('/content/drive/My Drive/Grupos_palavras_portugues.txt', 'a')
    arquivo.write(str(grupoAux)) 
    arquivo.write('\n')
    arquivo.close()



De uma maneira geral os resultados mostram a obtenção de agrupamentos condizentes com a prposta. Uma abordagem interessante seria limpar os dados incoerentes (termos nitidamente antagônicos como as palavras antônimas, por exemplo, que são observados em alguns casos de grupos gerados). Outra ideia seria extrair todas as palavras óbvias (aquelas que apresentam o mesmo radical e portanto, possuem semântica muito parecida) restando somente aquelas que possuem significado parecido, mas uma escrita notoriamente distinta. 

Optou-se por não desmembrar os termos compostos da lista de entrada original (mais de uma palavra). A ideia foi preservar o forte significado que certos termos juntos podem apresentar. Apesar de alguns ajuntamentos um tanto estranhos (antônimos), os resultados mostram agrupamentos interessantes. Por exemplo, as expressões 'really excellent', 'quite good' e 'truly great' que são sinônimas praticamente e escritas muito diferentes foram colocadas num mesmo agrupamento.  

Os resultados para a lista de palavras em português não foram tão bons...a razão é que o modelo versão em português da biblioteca spaCy disponível para download não possui vetores incorporados até o momento, dea cordo com a documentação da ferramenta. Desse modo o resultado dos cálculos de similaridade do algoritmo fica prejudicado.